In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

# Load the dataset
df = pd.read_csv("AI_Human.csv")  # Replace "your_dataset.csv"

Matplotlib is building the font cache; this may take a moment.


### Initial Data Inspection
##### 1. Shape and Size
##### 2. Missing Values
##### 3. First Few Rows
##### 4. Descriptive Statistics

In [ ]:
print(df.shape) # Shape and Size
print(df.dtypes) # Data Types of each column
print(df.isnull().sum())  # Missing values per column
print(df.head()) # First few rows
print(df.describe()) # Descriptive Statistics

(487235, 2)
text          object
generated    float64
dtype: object
text         0
generated    0
dtype: int64
                                                text  generated
0  Cars. Cars have been around since they became ...        0.0
1  Transportation is a large necessity in most co...        0.0
2  "America's love affair with it's vehicles seem...        0.0
3  How often do you ride in a car? Do you drive a...        0.0
4  Cars are a wonderful thing. They are perhaps o...        0.0
           generated
count  487235.000000
mean        0.372383
std         0.483440
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000


### Text-specific EDA 

#### Basic Text Statistics:
##### 1. Document Length: Calculate the length of each text (number of characters, number of words, number of sentences).
##### 2. Average Word Length: Calculate the average length of words in each text.
##### 3. Vocabulary Size: Determine the number of unique words in each text and in the entire dataset.
##### 4. Sentence Length: Calculate the number of words per sentence.

In [7]:
df['char_count'] = df['text'].str.len()
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
#df['sentence_count'] = df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

def avg_word_len(text):
    words = text.split()
    return sum(len(word) for word in words) / len(words) if len(words) > 0 else 0

df['avg_word_length'] = df['text'].apply(avg_word_len)

print(df[['char_count', 'word_count', 'avg_word_length']].describe())

# Vocabulary size (number of unique words)
all_words = ' '.join(df['text']).split()
unique_words = set(all_words)
print(f"Vocabulary size: {len(unique_words)}")

          char_count     word_count  avg_word_length
count  487235.000000  487235.000000    487235.000000
mean     2269.586592     393.096214         4.755985
std       988.814028     168.593328         0.521039
min         1.000000       0.000000         0.000000
25%      1583.000000     278.000000         4.415595
50%      2102.000000     363.000000         4.685144
75%      2724.000000     471.000000         5.020000
max     18322.000000    1668.000000       126.000000
Vocabulary size: 544828
